In [1]:
# !python -m venv fresh_env
# !pip install --upgrade pip setuptools wheel
# !pip install -r requirements.txt

In [2]:
# # In Jupyter/Colab: use %pip so it installs into the current kernel
# %pip uninstall -y tensorflow keras protobuf
# %pip install -U "tensorflow==2.12.1" "keras==2.12.0" "protobuf==3.20.3"

In [3]:
# !pip install "tensorflow==2.12.1" "keras==2.12.0" "protobuf==3.20.3"

In [1]:
# ===== STEP 2: After kernel restart, run this main code =====

##############################################
# === MELD Emotion Classification (Dependency-Free) ===
##############################################

import os
import pandas as pd
import numpy as np
import re
from collections import Counter
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional


# print("TensorFlow version:", tf.__version__)

# ---- Paths & labels ----
# Point to the correct directory
DATA_DIR = "/home/jupyter/old_backup"

TRAIN_CSV = os.path.join(DATA_DIR, "train_sent_emo.csv")
DEV_CSV = os.path.join(DATA_DIR, "dev_sent_emo.csv")
TEST_CSV = os.path.join(DATA_DIR, "test_sent_emo.csv")

CLASSES = ["anger", "disgust", "fear", "joy", "neutral", "sadness", "surprise"]
label2id = {c: i for i, c in enumerate(CLASSES)}
id2label = {i: c for c, i in label2id.items()}

2025-08-27 15:12:09.096404: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
# # ---- Load data ----
def load_meld_data():
    """Load and preprocess MELD dataset"""
    use_cols = ["Utterance", "Emotion"]

    try:
        train_df = pd.read_csv(TRAIN_CSV)[use_cols].dropna()
        dev_df = pd.read_csv(DEV_CSV)[use_cols].dropna()
        test_df = pd.read_csv(TEST_CSV)[use_cols].dropna()
    except FileNotFoundError as e:
        print(f"Error loading data: {e}")
        print("Please ensure the MELD CSV files are in the correct directory")
        return None, None, None

    # Map emotions to labels

    for name, df in zip(("train", "dev", "test"), (train_df, dev_df, test_df)):
        df["label"] = df["Emotion"].map(label2id)
        df.dropna(subset=["label"], inplace=True)
        df.rename(columns={"Utterance": "text"}, inplace=True)

    print(f"Loaded - Train: {len(train_df)}, Dev: {len(dev_df)}, Test: {len(test_df)}")

    # Check label distribution
    print("\nLabel distribution in training data:")
    label_counts = train_df["Emotion"].value_counts()
    for emotion, count in label_counts.items():
        print(f"  {emotion}: {count}")

    return train_df, dev_df, test_df

In [3]:
train_df, dev_df, test_df = load_meld_data()

if train_df is None:
    print("Data loading failed. Please check your file paths.")
    exit()


# ---- Text preprocessing ----
def clean_text(text):
    """Clean and normalize text"""
    if pd.isna(text):
        return ""

    text = str(text).lower()
    # Keep basic punctuation that might be emotionally relevant
    text = re.sub(r"[^\w\s!?.,]", " ", text)
    text = re.sub(r"\s+", " ", text)
    return text.strip()


# Apply preprocessing
train_df["text_clean"] = train_df["text"].apply(clean_text)
dev_df["text_clean"] = dev_df["text"].apply(clean_text)
test_df["text_clean"] = test_df["text"].apply(clean_text)

Loaded - Train: 9989, Dev: 1109, Test: 2610

Label distribution in training data:
  neutral: 4710
  joy: 1743
  surprise: 1205
  anger: 1109
  sadness: 683
  disgust: 271
  fear: 268


In [4]:
# !pip install tensorflow==2.16.2 keras==3.3.3 protobuf==4.25.3 --force-reinstall

In [5]:
train_df = train_df[train_df["text_clean"].str.len() > 0]
dev_df = dev_df[dev_df["text_clean"].str.len() > 0]
test_df = test_df[test_df["text_clean"].str.len() > 0]

print(
    f"After cleaning - Train: {len(train_df)}, Dev: {len(dev_df)}, Test: {len(test_df)}"
)

# ---- Tokenization ----
# Create vocabulary from training data
vocab_size = 10000
max_len = 128

tokenizer = Tokenizer(num_words=vocab_size, oov_token="<OOV>", lower=True, split=" ")

print("Building vocabulary...")
tokenizer.fit_on_texts(train_df["text_clean"])

print(f"Vocabulary size: {len(tokenizer.word_index)}")
print(f"Most common words: {list(tokenizer.word_index.items())[:10]}")

# Convert texts to sequences
X_train = tokenizer.texts_to_sequences(train_df["text_clean"])
X_dev = tokenizer.texts_to_sequences(dev_df["text_clean"])
X_test = tokenizer.texts_to_sequences(test_df["text_clean"])

# Pad sequences
X_train_pad = pad_sequences(X_train, maxlen=max_len, padding="post", truncating="post")
X_dev_pad = pad_sequences(X_dev, maxlen=max_len, padding="post", truncating="post")
X_test_pad = pad_sequences(X_test, maxlen=max_len, padding="post", truncating="post")

After cleaning - Train: 9989, Dev: 1109, Test: 2610
Building vocabulary...
Vocabulary size: 5364
Most common words: [('<OOV>', 1), ('i', 2), ('you', 3), ('the', 4), ('s', 5), ('it', 6), ('to', 7), ('a', 8), ('that', 9), ('and', 10)]


In [9]:
# Get labels

y_train = train_df["label"].values
y_dev = dev_df["label"].values
y_test = test_df["label"].values

print(
    f"Final shapes - Train: {X_train_pad.shape}, Dev: {X_dev_pad.shape}, Test: {X_test_pad.shape}"
)


# ---- Model Architecture ----
def create_emotion_model(vocab_size, embedding_dim=128, max_len=128, num_classes=7):
    """Create the emotion classification model"""
    model = Sequential(
        [
            # Embedding layer
            Embedding(
                input_dim=vocab_size,
                output_dim=embedding_dim,
                input_length=max_len,
                mask_zero=True,
            ),
            # Bidirectional LSTM layers
            Bidirectional(
                LSTM(64, return_sequences=True, dropout=0.3, recurrent_dropout=0.3)
            ),
            Bidirectional(
                LSTM(32, return_sequences=False, dropout=0.3, recurrent_dropout=0.3)
            ),
            # Dense layers
            Dense(64, activation="relu"),
            Dropout(0.5),
            Dense(32, activation="relu"),
            Dropout(0.3),
            Dense(num_classes, activation="softmax"),
        ]
    )
    # Dense(128, activation='relu'),
    # Dropout(0.5),
    # Dense(64, activation='relu'),
    # Dropout(0.4),
    # Dense(32, activation='relu'),
    # Dropout(0.3),
    # Dense(num_classes, activation='softmax')])

    return model


# Create model
actual_vocab_size = min(vocab_size, len(tokenizer.word_index) + 1)
model = create_emotion_model(
    vocab_size=actual_vocab_size,
    embedding_dim=128,
    max_len=max_len,
    num_classes=len(CLASSES),
)

# Compile model
model.compile(
    optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"]
)

model.summary()

Final shapes - Train: (9989, 128), Dev: (1109, 128), Test: (2610, 128)
Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 128, 128)          686720    
                                                                 
 bidirectional_2 (Bidirectio  (None, 128, 128)         98816     
 nal)                                                            
                                                                 
 bidirectional_3 (Bidirectio  (None, 64)               41216     
 nal)                                                            
                                                                 
 dense_4 (Dense)             (None, 64)                4160      
                                                                 
 dropout_3 (Dropout)         (None, 64)                0         
                                                 

In [10]:
# ---- Training callbacks ----
callbacks = [
    EarlyStopping(
        monitor="val_accuracy", patience=5, restore_best_weights=True, verbose=1
    ),
    ReduceLROnPlateau(
        monitor="val_loss", factor=0.5, patience=3, min_lr=1e-7, verbose=1
    ),
]

In [11]:
# ---- Train model ----
print("Starting training...")
history = model.fit(
    X_train_pad,
    y_train,
    batch_size=32,
    epochs=20,
    validation_data=(X_dev_pad, y_dev),
    callbacks=callbacks,
    verbose=1,
)

# ---- Evaluation ----
print("\nEvaluating on test set...")
test_loss, test_accuracy = model.evaluate(X_test_pad, y_test, verbose=0)
print(f"Test Accuracy: {test_accuracy:.4f}")

Starting training...
Epoch 1/20
313/313 [==============================] - 323s 945ms/step - loss: 1.5886 - accuracy: 0.4720 - val_loss: 1.5523 - val_accuracy: 0.4617 - lr: 0.0010
Epoch 2/20
313/313 [==============================] - 281s 899ms/step - loss: 1.4278 - accuracy: 0.5087 - val_loss: 1.5178 - val_accuracy: 0.4572 - lr: 0.0010
Epoch 3/20
313/313 [==============================] - 289s 923ms/step - loss: 1.3189 - accuracy: 0.5246 - val_loss: 1.5537 - val_accuracy: 0.4626 - lr: 0.0010
Epoch 4/20
313/313 [==============================] - 313s 999ms/step - loss: 1.2062 - accuracy: 0.5676 - val_loss: 1.5813 - val_accuracy: 0.4436 - lr: 0.0010
Epoch 5/20
313/313 [==============================] - ETA: 0s - loss: 1.1045 - accuracy: 0.6082
Epoch 5: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
313/313 [==============================] - 278s 886ms/step - loss: 1.1045 - accuracy: 0.6082 - val_loss: 1.7317 - val_accuracy: 0.4436 - lr: 0.0010
Epoch 6/20
313/313 [===

In [ ]:
# Predictions
y_pred = model.predict(X_test_pad, verbose=0)
y_pred_classes = np.argmax(y_pred, axis=1)


# ---- Classification Report (Manual Implementation) ----
def classification_report_manual(y_true, y_pred, class_names):
    """Manual classification report to avoid sklearn dependency"""

    # Calculate metrics for each class
    report = {}

    for i, class_name in enumerate(class_names):
        # True positives, false positives, false negatives
        tp = np.sum((y_true == i) & (y_pred == i))
        fp = np.sum((y_true != i) & (y_pred == i))
        fn = np.sum((y_true == i) & (y_pred != i))

        # Calculate precision, recall, f1
        precision = tp / (tp + fp) if (tp + fp) > 0 else 0
        recall = tp / (tp + fn) if (tp + fn) > 0 else 0
        f1 = (
            2 * (precision * recall) / (precision + recall)
            if (precision + recall) > 0
            else 0
        )

        support = np.sum(y_true == i)

        report[class_name] = {
            "precision": precision,
            "recall": recall,
            "f1-score": f1,
            "support": support,
        }

    return report


# Generate manual classification report
report = classification_report_manual(y_test, y_pred_classes, CLASSES)

print("\n" + "=" * 60)
print("CLASSIFICATION REPORT")
print("=" * 60)
print(f"{'Class':<12} {'Precision':<10} {'Recall':<10} {'F1-Score':<10} {'Support':<8}")
print("-" * 60)

for class_name, metrics in report.items():
    print(
        f"{class_name:<12} {metrics['precision']:<10.4f} {metrics['recall']:<10.4f} "
        f"{metrics['f1-score']:<10.4f} {metrics['support']:<8}"
    )

# Overall accuracy
overall_accuracy = np.mean(y_test == y_pred_classes)
print(f"\nOverall Accuracy: {overall_accuracy:.4f}")

In [ ]:
# ---- Plot training history (if matplotlib available) ----
def plot_history(history):
    """Plot training history"""
    try:
        import matplotlib.pyplot as plt

        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))

        # Accuracy plot
        ax1.plot(history.history["accuracy"], label="Training")
        ax1.plot(history.history["val_accuracy"], label="Validation")
        ax1.set_title("Model Accuracy")
        ax1.set_xlabel("Epoch")
        ax1.set_ylabel("Accuracy")
        ax1.legend()
        ax1.grid(True)

        # Loss plot
        ax2.plot(history.history["loss"], label="Training")
        ax2.plot(history.history["val_loss"], label="Validation")
        ax2.set_title("Model Loss")
        ax2.set_xlabel("Epoch")
        ax2.set_ylabel("Loss")
        ax2.legend()
        ax2.grid(True)

        plt.tight_layout()
        plt.show()

    except ImportError:
        print("Matplotlib not available - skipping plots")


plot_history(history)

print(
    f"\nTraining completed! Best validation accuracy: {max(history.history['val_accuracy']):.4f}"
)

# ---- Sample predictions ----
print("\n" + "=" * 60)
print("SAMPLE PREDICTIONS")
print("=" * 60)

# Show a few test examples
sample_indices = np.random.choice(len(test_df), min(5, len(test_df)), replace=False)

for idx in sample_indices:
    original_idx = test_df.index[idx]
    text = test_df.loc[original_idx, "text"]
    true_label = y_test[idx]
    pred_label = y_pred_classes[idx]
    confidence = np.max(y_pred[idx])

    print(f"\nText: {text}")
    print(
        f"True: {CLASSES[true_label]} | Predicted: {CLASSES[pred_label]} | Confidence: {confidence:.3f}"
    )

print("\n" + "=" * 60)